In [1]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

class RewardAndGoalLoggingCallback(BaseCallback):
    def __init__(self, log_interval=20000, verbose=0):
        super().__init__(verbose)
        self.log_interval = log_interval
        self.cumulative_reward = 0
        self.step_counter = 0

    def _on_step(self):
        reward = self.locals["rewards"][0]
        self.cumulative_reward += reward
        self.step_counter += 1

        if self.step_counter % self.log_interval == 0:
            # Get the goal_pos from the environment (VecEnv-safe)
            goal_pos = self.training_env.get_attr("goal_pos")[0]
            ef_pos = self.training_env.get_attr("robot")[0].get_joint_obs()["ee_pos"]
            print(f"📈 Step {self.num_timesteps} | Reward (last {self.log_interval}): {round(self.cumulative_reward, 4)} | 🎯 Goal: {np.round(goal_pos, 3)},f🤖 EE Pos: {np.round(ef_pos, 3)} ")
            self.cumulative_reward = 0

        return True

In [2]:
from stable_baselines3 import PPO
from env_2 import UArmEnv

# Create environment and model
env = UArmEnv(render=False)
model = PPO(
    "MlpPolicy",
    env,
    ent_coef=0.005,        # Encourages broader exploration without overwhelming reward
    n_steps=2048,          # Longer rollout → better credit assignment
    batch_size=256,        # Good match for long rollouts
    gae_lambda=0.95,       # Slightly longer-term reward tracking
    gamma=0.99,            # Standard long-term discount
    verbose=1
)

# Add logging callback
reward_logger = RewardAndGoalLoggingCallback(log_interval=20000)

# Train with logging
model.learn(total_timesteps=1_000_000, callback=reward_logger)

env.close()

model.save("ppo_uarm_final_25")


pybullet build time: Jan 29 2025 23:16:28


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/fatduck/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/fatduck/.local/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


⏱️ Timeout at Step 500 | EE: [-0.164 -0.184  0.06 ] | Goal: [-0.169 -0.248  0.03 ]
⏱️ Timeout at Step 500 | EE: [ 0.117 -0.174  0.234] | Goal: [-0.169 -0.248  0.03 ]
⏱️ Timeout at Step 500 | EE: [ 0.011 -0.16   0.161] | Goal: [-0.169 -0.318  0.03 ]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 488      |
|    ep_rew_mean     | -122     |
| time/              |          |
|    fps             | 544      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
⏱️ Timeout at Step 500 | EE: [ 0.021 -0.179  0.101] | Goal: [-0.169 -0.318  0.03 ]
⏱️ Timeout at Step 500 | EE: [-0.14  -0.209  0.01 ] | Goal: [-0.169 -0.248  0.03 ]
⏱️ Timeout at Step 500 | EE: [-0.04  -0.079  0.25 ] | Goal: [-0.169 -0.248  0.03 ]
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 452         |
|    ep_rew_mean          | -112    

KeyboardInterrupt: 

In [ ]:
model.save("ppo_uarm")
1 000 000